In [ ]:
!pip install transformers datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl 

In [ ]:
# load data
from datasets import load_dataset

dataset = load_dataset("adrienheymans/imdb-movie-genres")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/54214 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/54200 [00:00<?, ? examples/s]

In [ ]:
dataset['train']

Dataset({
    features: ['title', 'text', 'genre', 'label'],
    num_rows: 54214
})

In [ ]:
# get unique genres
unique_genres = set(dataset['train']['genre'])
unique_genres

{'action',
 'adult',
 'adventure',
 'animation',
 'biography',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'family',
 'fantasy',
 'game-show',
 'history',
 'horror',
 'music',
 'musical',
 'mystery',
 'news',
 'reality-tv',
 'romance',
 'sci-fi',
 'short',
 'sport',
 'talk-show',
 'thriller',
 'war',
 'western'}

In [ ]:
# get the amount of classes
classes = len(unique_genres)
classes

27

In [ ]:
# get mapping from the dataset
genre2label = {}

for genre, label in zip(dataset['train']['genre'], dataset['train']['label']):
    genre2label[genre] = label

print(genre2label)

{'drama': 0, 'thriller': 1, 'adult': 2, 'documentary': 3, 'comedy': 4, 'crime': 5, 'reality-tv': 6, 'horror': 7, 'sport': 8, 'animation': 9, 'action': 10, 'fantasy': 11, 'short': 12, 'sci-fi': 13, 'music': 14, 'adventure': 15, 'talk-show': 16, 'western': 17, 'family': 18, 'mystery': 19, 'history': 20, 'news': 21, 'biography': 22, 'romance': 23, 'game-show': 24, 'musical': 25, 'war': 26}


In [ ]:
label2genre = {}

for label, genre in zip(dataset['train']['label'], dataset['train']['genre']):
    label2genre[label] = genre

print(label2genre)

{0: 'drama', 1: 'thriller', 2: 'adult', 3: 'documentary', 4: 'comedy', 5: 'crime', 6: 'reality-tv', 7: 'horror', 8: 'sport', 9: 'animation', 10: 'action', 11: 'fantasy', 12: 'short', 13: 'sci-fi', 14: 'music', 15: 'adventure', 16: 'talk-show', 17: 'western', 18: 'family', 19: 'mystery', 20: 'history', 21: 'news', 22: 'biography', 23: 'romance', 24: 'game-show', 25: 'musical', 26: 'war'}


In [ ]:
# tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, max_length=512)

In [ ]:
tokenized_genre = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/54214 [00:00<?, ? examples/s]

Map:   0%|          | 0/54200 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=classes, id2label=label2genre, label2id=genre2label
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
training_args = TrainingArguments(
    output_dir="movie_genre_multi_classification",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_genre["train"],
    eval_dataset=tokenized_genre["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.191100,5.534198,0.032066
2,0.951000,6.097394,0.030443


TrainOutput(global_step=6778, training_loss=1.162502069324123, metrics={'train_runtime': 4680.3254, 'train_samples_per_second': 23.167, 'train_steps_per_second': 1.448, 'total_flos': 9564487266978072.0, 'train_loss': 1.162502069324123, 'epoch': 2.0})

In [ ]:
text = """
In a post-apocalyptic world where humanity teeters on the brink of extinction, a lone scientist discovers a way to send messages back in time. Desperate to alter the course of history and prevent the cataclysm that brought about their downfall, she sends a series of cryptic messages to her past self. As she navigates the dangerous landscape of her present, she races against time to decipher the clues hidden within her own memories and unlock the key to humanity's salvation. But with each message sent, the fabric of time grows more fragile, and she must confront the consequences of meddling with the past. "Echoes of Eternity" is a gripping sci-fi thriller that explores the complexities of time travel, the resilience of the human spirit, and the enduring power of hope in the face of despair.
"""


In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification", model="handler-bird/movie_genre_multi_classification")
classifier(text)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

[{'label': 'sci-fi', 'score': 0.3335624039173126}]